In [42]:
from ipywidgets import interact, interact_manual, widgets, Label
import numpy as np
import functools
from time import time, sleep
import matplotlib.pyplot as plt

In [43]:
def generate_biased_rnd_vector_bits(p_in,n_in):
    v_raw_rnd_bin=np.array(np.random.rand(n_in)<p_in,dtype=np.uint8)
    return v_raw_rnd_bin

In [44]:
raw_bytes=widgets.Textarea(value='',placeholder='',description='',disabled=True,layout=widgets.Layout(width='200px',height='200px'))
tests_results=widgets.HTML(value='',placeholder='',description='',disabled=True, continuous_update=True,layout=widgets.Layout(width='200px',height='300px'))

nb_p=10000
v_p=0.5*np.arange(1,nb_p+1,dtype=np.float64)/(nb_p+1)
v_h=-1*(np.log2(v_p)*(v_p)+np.log2(1-v_p)*(1-v_p))

def errorsont1(entropy_in=0.5,test_size=512,Th_low=250,Th_high=260):
  nb_tests=100

  if entropy_in==1:
    p_target=0.5
  elif entropy_in==0:
    p_target=np.random.randint(0,2)
  else:
    rnd=np.random.randint(0,2)
    if rnd==0:
      p_target=v_p[np.where(v_h>entropy_in)[0][0]]
    else:
      p_target=1-v_p[np.where(v_h>entropy_in)[0][0]]

  v_in=generate_biased_rnd_vector_bits(p_target,int(nb_tests*test_size))

  s_line_raw=''
  for i in range(test_size*2):
    s_line_raw='%s%d'%(s_line_raw,v_in[i])
  raw_bytes.value=s_line_raw

  m_in=v_in.reshape(nb_tests,test_size)
  v_T1=m_in.sum(1)


  nb_alarms=((v_T1<Th_low) | (v_T1>Th_high)).sum()


  s_label_test='<p>%d alarms</p>'%(nb_alarms)
  tests_results.value=s_label_test
  

im=interact(errorsont1)
im.widget.close()
im.widget.children[0].max=1.0
im.widget.children[0].min=0.0
im.widget.children[0].step=0.01
im.widget.children[1].max=1024
im.widget.children[1].min=8
im.widget.children[1].step=1
im.widget.children[2].max=1024
im.widget.children[2].min=0
im.widget.children[2].step=1
im.widget.children[3].max=1024
im.widget.children[3].min=0
im.widget.children[3].step=1
im.widget.children[0].description='Entropy'
im.widget.children[1].description='Latency'
im.widget.children[2].description='Threshold Low'
im.widget.children[3].description='Threshold High'
v_box_top=widgets.VBox([im.widget.children[0],im.widget.children[1],im.widget.children[2],im.widget.children[3],widgets.HBox([raw_bytes,tests_results])])
display(v_box_top)

interactive(children=(FloatSlider(value=0.5, description='entropy_in', max=1.5, min=-0.5), IntSlider(value=512…